In [1]:
import yfinance as yf
from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATS
import pandas as pd
from datetime import datetime, timedelta

/Users/johtorr/Repos/llm-local-rag/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-14 16:55:13,596	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-11-14 16:55:13,639	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
def get_stock_forecast(stock_symbol: str, max_prediction_length : int = 10):

    max_encoder_length = 6
    FREQUENCY='D'
    #max_prediction_length = 10 # 5 days in the future
    val_size = 10 # 4 Months forecast horizon in validation

    # Download historical stock data from Yahoo Finance
    #stock_symbol='TSLA'

    stock_data = yf.download(stock_symbol, period="1y", interval="1d")

    # Prepare data for the model (assuming the model needs certain columns and format)
    stock_data = stock_data.reset_index().rename(columns={"Date": "ds", "Close": "y"})
    stock_data = stock_data[["ds", "y"]].copy()
    stock_data.columns = ["ds", "y"]
    stock_data['unique_id'] = stock_symbol

    models = [
        NBEATS(
            input_size=max_encoder_length,
            h=max_prediction_length,
            max_steps=max_encoder_length,
        ),
    ]

    # Initialize the NEBATS model for forecasting
    nf = NeuralForecast(models=models, freq=FREQUENCY)
    nf.fit(df=stock_data, val_size=val_size, verbose=False)
    future_dates = nf.predict().reset_index()
    future_dates = future_dates.rename(columns={'unique_id':'ticker', 'ds':'date', 'NBEATS':'prediction'})

    # Format the forecast results
    return future_dates


In [3]:
get_stock_forecast('TSLA')

[*********************100%***********************]  1 of 1 completed
Seed set to 1
/Users/johtorr/Repos/llm-local-rag/.venv/lib/python3.12/site-packages/neuralforecast/common/_base_model.py:345: UserWarning: val_check_steps is greater than max_steps, setting val_check_steps to max_steps.
  warnings.warn(
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MAE           | 0      | train
1 | padder_train | ConstantPad1d | 0      | train
2 | scaler       | TemporalNorm  | 0      | train
3 | blocks       | ModuleList    | 2.4 M  | train
-------------------------------------------------------
2.4 M     Trainable params
336       Non-trainable params
2.4 M     Total params
9.619     Total estimated model params size (MB)
31        Modules in train mode
0         Modules in eval mode


Epoch 5: 100%|██████████| 1/1 [00:00<00:00, 26.10it/s, v_num=16, train_loss_step=17.40, train_loss_epoch=17.40, valid_loss=49.70]

`Trainer.fit` stopped: `max_steps=6` reached.


Epoch 5: 100%|██████████| 1/1 [00:00<00:00, 25.10it/s, v_num=16, train_loss_step=17.40, train_loss_epoch=17.40, valid_loss=49.70]

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs



Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 133.53it/s]


/Users/johtorr/Repos/llm-local-rag/.venv/lib/python3.12/site-packages/neuralforecast/core.py:210: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


,ticker,date,prediction
0,TSLA,2024-11-15 00:00:00+00:00,328.103180
1,TSLA,2024-11-16 00:00:00+00:00,297.503204
2,TSLA,2024-11-17 00:00:00+00:00,319.909546
3,TSLA,2024-11-18 00:00:00+00:00,322.592926
4,TSLA,2024-11-19 00:00:00+00:00,306.447601
5,TSLA,2024-11-20 00:00:00+00:00,330.680878
6,TSLA,2024-11-21 00:00:00+00:00,336.297424
7,TSLA,2024-11-22 00:00:00+00:00,326.578674
8,TSLA,2024-11-23 00:00:00+00:00,310.416016
9,TSLA,2024-11-24 00:00:00+00:00,320.315735


In [8]:
import requests
import json
import os 

api_key = os.environ['OPENAI_API_KEY']
endpoint = "https://api.openai.com/v1/chat/completions"

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}

# Define the function schema for OpenAI function calling
function_schema = {
    "name": "get_stock_forecast",
    "description": "Get stock price forecast for the next five days",
    "parameters": {
        "type": "object",
        "properties": {
            "stock_symbol": {
                "type": "string",
                "description": "The stock symbol to forecast (e.g., 'AAPL', 'MSFT')."
            }
        },
        "required": ["stock_symbol"]
    }
}

# Define the initial user message
messages = [
    {"role": "user", "content": "Get stock price forecast for Qualcomm"}
]

# Prepare the request data with function calling
data = {
    "model": "gpt-4o-mini",  # or any other compatible model
    "messages": messages,
    "functions": [function_schema],
    "function_call": "auto"
}

In [9]:
# Send the request
response = requests.post(endpoint, headers=headers, data=json.dumps(data))

# Check if the response includes a function call request
if response.status_code == 200:
    result = response.json()
    function_call = result["choices"][0].get("message", {}).get("function_call")

    if function_call:
        # If the model requests a function call, extract arguments and call the function
        arguments = json.loads(function_call["arguments"])
        stock_symbol = arguments["stock_symbol"]
        forecast = get_stock_forecast(stock_symbol)
        print(f"Forecast for {stock_symbol}:", forecast)
    else:
        print(result["choices"][0]["message"]["content"])
else:
    print(f"Request failed with status code {response.status_code}: {response.text}")

[*********************100%***********************]  1 of 1 completed
Seed set to 1
/Users/johtorr/Repos/llm-local-rag/.venv/lib/python3.12/site-packages/neuralforecast/common/_base_model.py:345: UserWarning: val_check_steps is greater than max_steps, setting val_check_steps to max_steps.
  warnings.warn(
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MAE           | 0      | train
1 | padder_train | ConstantPad1d | 0      | train
2 | scaler       | TemporalNorm  | 0      | train
3 | blocks       | ModuleList    | 2.4 M  | train
-------------------------------------------------------
2.4 M     Trainable params
336       Non-trainable params
2.4 M     Total params
9.619     Total estimated model params size (MB)
31        Modules in train mode
0         Modules in eval mode


Epoch 5: 100%|██████████| 1/1 [00:00<00:00, 26.94it/s, v_num=20, train_loss_step=13.80, train_loss_epoch=13.80, valid_loss=8.070]

`Trainer.fit` stopped: `max_steps=6` reached.


Epoch 5: 100%|██████████| 1/1 [00:00<00:00, 25.99it/s, v_num=20, train_loss_step=13.80, train_loss_epoch=13.80, valid_loss=8.070]


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 234.70it/s]
Forecast for QCOM:   ticker                      date  prediction
0   QCOM 2024-11-15 00:00:00+00:00  159.472488
1   QCOM 2024-11-16 00:00:00+00:00  150.782288
2   QCOM 2024-11-17 00:00:00+00:00  171.692551
3   QCOM 2024-11-18 00:00:00+00:00  182.642395
4   QCOM 2024-11-19 00:00:00+00:00  148.371658
5   QCOM 2024-11-20 00:00:00+00:00  172.857285
6   QCOM 2024-11-21 00:00:00+00:00  172.897049
7   QCOM 2024-11-22 00:00:00+00:00  175.358490
8   QCOM 2024-11-23 00:00:00+00:00  159.263351
9   QCOM 2024-11-24 00:00:00+00:00  164.023407


/Users/johtorr/Repos/llm-local-rag/.venv/lib/python3.12/site-packages/neuralforecast/core.py:210: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


In [51]:
function_call

{'name': 'get_stock_forecast', 'arguments': '{"stock_symbol":"MBG.DE"}'}